In [4]:
import time
from datetime import datetime

current_time = time.time()
current_date = datetime.utcfromtimestamp(current_time).strftime('%Y-%m-%d')

print("Current date:", current_date)

Current date: 2024-05-06


In [5]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

openai_api_key = os.getenv("OPENAI_API_KEY")
chat = ChatOpenAI(
    temperature=0.2,
    model_name="gpt-3.5-turbo",
    openai_api_key=openai_api_key,
    max_tokens=100,
)

user = "Mike"

# Create a single memory instance for the entire conversation
memory = ConversationBufferMemory(return_messages=True)

def yes_no_response(input_message, model, memory):
    prompt = ChatPromptTemplate.from_messages([
        ("system", f""" Your task is to identify if the {user} asking for a documents/applications/forms for annual leave, casual leave, sick leave, self assesment, time off after analyzing the whole conversation.
                        You have to understand the context of the conversation and then generate the response in these forms annual leave, casual leave, sick leave, self assesment, time off, mobile claim, petrol claim.
                        Your response should be either "annual leave", "casual leave", "sick leave", "self assesment", "time off", "mobile claim", "petrol claim" on every query you will get.
                        Stick to this professional tone and provide simple, two-word answers.

                        Here are the examples I want you to strictly follow:

                        Conversation Example 1:
                        User: Hi.
                        Your response: false

                        User: I want a leave
                        Your response: false
         
                        User: 15 July to 19 July
                        Your response: false
         
                        User: How can I apply for it?
                        Your response: false
         
                        User: How can I apply for it?
                        Your response: false
         
                        User: Do you have a template for sick leave?
                        Your response: sick leave

                        ------------------------------------------

                        Conversation Example 2:
                        User: Hi there
                        Your response: false

                        User: Do you have leave data available?
                        Your response: false
         
                        User: Am I eligible for a leave?
                        Your response: false
         
                        User: I need a leave in august from 5 to 9.
                        Your response: false
         
                        User: Can you guide me through the leave process?
                        Your response: false
         
                        User: I want a proposal/document to apply for a casual leave.
                        Your response: casual leave

                        ------------------------------------------

                        Conversation Example 3:
                        User: Hello, I'm John.
                        Your response: false

                        User: I want to go on a vacation.
                        Your response: false
         
                        User: 18 sept - 27 sept
                        Your response: false
         
                        User: Can you tell me that these dates are available?
                        Your response: false
         
                        User: What are the steps to apply for annual leave.
                        Your response: false
         
                        User: I want to apply for it.
                        Your response: false
         
                        User: write me an application.
                        Your response: annual leave

                        ------------------------------------------

                        Conversation Example 4:
                        User: Hello!
                        Your response: false
         
                        User: Can you check my leave entitlement?
                        Your response: false
         
                        User: Can I plan a trip in August?
                        Your response: false
         
                        User: Can you assist with the casual leave request process?
            `           Your response: false
         
                        User: Can you draft a leave application email for me?
                        Your response: casual leave
         
                        ------------------------------------------
         
                        Conversation Example 5:
                        User: Hi!
                        Your response: false

                        User: Do I have any leave days left?
                        Your response: false

                        User: Can I take time off in August?
                        Your response: false

                        User: Can you help me write an email to request time off form?
                        Your response: time off form
         
                        ------------------------------------------
         
                        Conversation Example 6:
                        User: Who are you?
                        Your response: false

                        User: Do I have any leave days left?
                        Your response: false

                        User: Can I take time off in August?
                        Your response: false

                        User: Step by step procedure for to apply for a leave.
                        Your response: false
         
                        User: Do you have a form for a leave.
                        Your response: time off form
                        ------------------------------------------
         
                        User: Hi, I'm Sarah.
                        Your response: false

                        User: I wanted to discuss my performance appraisal.
                        Your response: false

                        User: Can you tell me how I can evaluate my performance?
                        Your response: false

                        User: I heard about a self-assessment form. Can you guide me on how to fill it out?
                        Your response: self assessment form
                        
                        ------------------------------------------

                        User: Hi, I'm Mark.
                        Your response: false

                        User: I need to claim petrol expenses for the job.
                        Your response: false

                        User: Could you please provide me with the petrol claim form?
                        Your response: petrol claim form
         
                        ------------------------------------------

                        User: Hi, I'm Emily.
                        Your response: false

                        User: I need to claim mobile expenses for the job.
                        Your response: false

                        User: Do you have a document in which I can fill the information and submit it to HR?
                        Your response: mobile claim form

                        ------------------------------------------

                        User: Hi, I'm Emily.
                        Your response: false

                        User: I need to claim medical expenses for the job.
                        Your response: false

                        User: Do you have a document in which I can fill the information and submit it to HR?
                        Your response: insurance claim form

                        You are strictly not allowed to say anything rather than these "annual leave", "casual leave", "sick leave", "self assesment", "time off", "mobile claim", "petrol claim".
                        """),
        MessagesPlaceholder(variable_name="history"),
        ("human", f"{input_message}"),
    ])

    chain = (
        RunnablePassthrough.assign(
            history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
        )
        | prompt
        | model
    )

    inputs = {"input": input_message}
    response = chain.invoke(inputs)

    # Save the context for future interactions
    memory.save_context(inputs, {"output": response.content})
    memory.load_memory_variables({})

    return response.content

In [6]:
human_messages = "Hi"
print(yes_no_response(human_messages, chat, memory))

false


In [7]:
human_messages = "Who are you?"
print(yes_no_response(human_messages, chat, memory))

false


In [8]:
human_messages = "Do you have any data related to leave?"
print(yes_no_response(human_messages, chat, memory))

false


In [9]:
human_messages = "How do I evaluate the performance?"
print(yes_no_response(human_messages, chat, memory))

false


In [10]:
human_messages = "Do you have any document?"
print(yes_no_response(human_messages, chat, memory))

self assessment
